### Annual Sales Tax Revenue Analysis

#### [Data Source](https://tax.utah.gov/econstats/sales)

#### Notes:


In [19]:
import arcpy
from arcpy import env
import os
import numpy as np
from arcgis import GIS
from arcgis.features import GeoAccessor
from arcgis.features import GeoSeriesAccessor
import pandas as pd
import glob

arcpy.env.overwriteOutput = True
arcpy.env.parallelProcessingFactor = "90%"

# show all columns
pd.options.display.max_columns = None

# pd.DataFrame.spatial.from_featureclass(???)  
# df.spatial.to_featureclass(location=???,sanitize_columns=False)  

# gsa = arcgis.features.GeoSeriesAccessor(df['SHAPE'])  
# df['AREA'] = gsa.area  # KNOW YOUR UNITS

In [31]:
if not os.path.exists('Outputs'):
    os.makedirs('Outputs')
    
outputs = ['.\\Outputs', 'Utah.gdb']
gdb = os.path.join(outputs[0], outputs[1])

if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs[0], outputs[1])


In [35]:
# read in cities as spatial dataframe
global cities_sdf
cities_sdf = pd.DataFrame.spatial.from_featureclass(r".\Inputs\Cities_v2.shp")
cities_sdf = cities_sdf[['NAME','POPLASTCEN','POPLASTEST', 'ACRES','SHAPE']].copy()
cities_sdf.rename({'NAME':'City'}, axis=1, inplace=True)

In [22]:
# dictionary for renaming tax revenue sectors
global sector_lu
sector_lu= {
              'ACCOMMODATION (721)':'L_ACCOMMODATION',
              'ADMIN. & SUPPORT & WASTE MANAG. & REMED. SERVICES (56)':'S_ADMIN_SUPPORT',
              'AGRICULTURE, FORESTRY, FISHING & HUNTING (11)':'A_AG_WILDLIFE',
              'ARTS, ENTERTAINMENT AND RECREATION (71)':'L_CULTURAL_REC', 
              'CONSTRUCTION (23)':'C_CONSTRUCTION',
              'EDUCATIONAL SERVICES (61)':'E_EDUCATION', 
              'FINANCE & INSURANCE (52)':'S_FINANCIAL',
              'FOOD SERVICES & DRINKING PLACES (722)':'L_RSTRNT_BAR',
              'HEALTH CARE & SOCIAL ASSISTANCE (62)':'H_HEALTH_CARE', 
              'INFORMATION (51)':'S_IT_DATA_20',
              'MANAGEMENT OF COMPANIES & ENTERPRISES (55)':'S_CORPORATE_MGMT', 
              'MANUFACTURING (31-33)':'M_MANUFACTURING',
              'MINING, QUARRYING, & OIL & GAS EXTRACTION (21)':'E_EXTRACTION',
              'OTHER SERVICES-EXCEPT PUBLIC ADMINISTRATION (81)':'O_OTHER',
              'PRIOR-PERIOD PAYMENTS & REFUNDS':'X_ADJUSTMENTS', 
              'PRIVATE MOTOR VEHICLE SALES':'R_AUTO_PRIVATE',
              'PROFESSIONAL, SCIENTIFIC & TECHNICAL SERVICES (54)':'S_PROF_TECH_SRV',
              'PUBLIC ADMINISTRATION (92)':'G_GOVERNMENT', 
              'REAL ESTATE, RENTAL & LEASING (53)':'S_RENTAL_LEASING',
              'RETAIL-BUILD. MATERIAL, GARDEN EQUIP. & SUPPLIES DEALERS (444)':'R_BUILDING_SUPPLY',
              'RETAIL-CLOTHING & CLOTHING ACCESSORIES STORES (448)':'R_CLOTHING',
              'RETAIL-CLOTHING & CLOTHING ACCESSORIES STORES (448,458)':'R_CLOTHING',
              'RETAIL-ELECTRONICS & APPLIANCE STORES (443)':'R_ELECTRONICS',
              'RETAIL-ELECTRONICS & APPLIANCE STORES (443,4492)':'R_ELECTRONICS',
              'RETAIL-FOOD & BEVERAGE STORES (445)':'R_GROCERY_BEV',
              'RETAIL-FURNITURE & HOME FURNISHINGS STORES (442)':'R_FURNITURE',
              'RETAIL-FURNITURE & HOME FURNISHINGS STORES (442,4491)':'R_FURNITURE',
              'RETAIL-GASOLINE STATIONS (447)':'R_GAS_STATIONS',
              'RETAIL-GASOLINE STATIONS (447,457)': 'R_GAS_STATIONS',
              'RETAIL-GENERAL MERCHANDISE STORES (452)':'R_GENERAL_RETAIL',
              'RETAIL-GENERAL MERCHANDISE STORES (452,455)':'R_GENERAL_RETAIL',
              'RETAIL-HEALTH & PERSONAL CARE STORES (446)':'R_HEALTH_RETAIL',
              'RETAIL-HEALTH & PERSONAL CARE STORES (446,456)':'R_HEALTH_RETAIL',
              'RETAIL-MISCELLANEOUS STORE RETAILERS (453)':'R_OTHER_RETAIL',
              'RETAIL-MISCELLANEOUS STORE RETAILERS (453,4593-4599)':'R_OTHER_RETAIL',
              'RETAIL-MOTOR VEHICLE & PARTS DEALERS (441)':'R_AUTO_RETAIL',
              'RETAIL-NONSTORE RETAILERS (454)':'R_NONSTORE_RETAIL',
              'RETAIL-SPORTING GOODS, HOBBY, MUSIC & BOOK STORES (451)':'R_SPORT_HOBBY',
              'RETAIL-SPORTING GOODS, HOBBY, MUSIC & BOOK STORES (451,4591,4592)':'R_SPORT_HOBBY',
              'SPECIAL EVENT SALES':'L_SPECIAL_EVENT', 
              'TRANSPORTATION & WAREHOUSING (48-49)':'W_DISTRIBUTION',
              'UNKNOWN/NONCLASSIFIABLE':'X_UNKNOWN', 
              'UTILITIES (22)':'U_UTILITIES',
              'WHOLESALE TRADE-DURABLE GOODS (423)':'W_WHLSALE_DURABLE',
              'WHOLESALE TRADE-ELECTRONIC MARKETS (425)':'S_WHLSLE_ETRADE',
              'WHOLESALE TRADE-AGENTS & BROKERS (425)':'S_WHLSLE_ETRADE',
              'WHOLESALE TRADE-NONDURABLE GOODS (424)':'W_WHLSLE_NDURABLE'            
}

In [23]:
# fill NA values in Spatially enabled dataframes (ignores SHAPE column)
def fill_na_sedf(df_with_shape_column, fill_value=0):
    if 'SHAPE' in list(df_with_shape_column.columns):
        df = df_with_shape_column.copy()
        shape_column = df['SHAPE'].copy()
        del df['SHAPE']
        return df.fillna(fill_value).merge(shape_column,left_index=True, right_index=True, how='inner')
    else:
        raise Exception("Dataframe does not include 'SHAPE' column")

In [26]:
def process_annual_sales_to_shape(_xlsx, _gdb):
    
    year = os.path.split(_xlsx)[-1][:4]
    tsr = pd.read_excel(_xlsx, sheet_name='Table 9', header=5)

    # Drop last empty two rows
    tsr.drop(tsr.tail(2).index,inplace=True)

    # forward fill values from merged cells
    tsr['County'] = tsr['County'].fillna(method='ffill')
    tsr['Location Code'] = tsr['Location Code'].fillna(method='ffill')
    tsr['City'] = tsr['City'].fillna(method='ffill')

    # Figures with less than 10 taxpayers have been rounded up per Tax Commission disclosure rules.
    # Rename column and convert values to boolean
    tsr.rename(columns={'Unnamed: 5':'Rounded_Up'}, inplace=True)
    crosswalk = {'*':True, np.nan:False}
    tsr['Rounded_Up'] = tsr['Rounded_Up'].map(crosswalk)

    # pivot the table to get sectors as columns with tax values by city
    tsr_by_sector = pd.pivot_table(tsr,values = f'CY{year}', index ='City', columns = 'Economic Sector (NAICS Code)', 
                                    aggfunc='first').reset_index()

    # fill NAs with 0
    tsr_by_sector = tsr_by_sector.fillna(0)

    # rename sectors
    _sector_lu = sector_lu
    _sector_lu = {k:v + f"_{year[-2:]}" for (k,v) in _sector_lu.items()}
    tsr_by_sector.rename(_sector_lu, axis=1, inplace=True)

    # get the total sales tax revenue
    tsr_by_sector['TOTAL'] = tsr_by_sector[_sector_lu.values()].sum(axis=1)

    # export
    tsr_by_sector_sdf = cities_sdf.merge(tsr_by_sector, on='City', how='inner')
    return tsr_by_sector_sdf.spatial.to_featureclass(location=os.path.join(_gdb,f'Taxable_Sales_by_City_{year}'),sanitize_columns=False)


In [38]:
def process_annual_sales_to_df(_xlsx):
    
    year = os.path.split(_xlsx)[-1][:4]
    tsr = pd.read_excel(_xlsx, sheet_name='Table 9', header=5)

    # Drop last empty two rows
    tsr.drop(tsr.tail(2).index,inplace=True)

    # forward fill values from merged cells
    tsr['County'] = tsr['County'].fillna(method='ffill')
    tsr['Location Code'] = tsr['Location Code'].fillna(method='ffill')
    tsr['City'] = tsr['City'].fillna(method='ffill')

    # Figures with less than 10 taxpayers have been rounded up per Tax Commission disclosure rules.
    # Rename column and convert values to boolean
    tsr.rename(columns={'Unnamed: 5':'Rounded_Up'}, inplace=True)
    crosswalk = {'*':True, np.nan:False}
    tsr['Rounded_Up'] = tsr['Rounded_Up'].map(crosswalk)

    # pivot the table to get sectors as columns with tax values by city
    tsr_by_sector = pd.pivot_table(tsr,values = f'CY{year}', index ='City', columns = 'Economic Sector (NAICS Code)', 
                                    aggfunc='first').reset_index()

    # fill NAs with 0
    tsr_by_sector = tsr_by_sector.fillna(0)

    # rename sectors
    _sector_lu = sector_lu
    _sector_lu = {k:v + f"_{year[-2:]}" for (k,v) in _sector_lu.items()}
    tsr_by_sector.rename(_sector_lu, axis=1, inplace=True)

    # get the total sales tax revenue
    tsr_by_sector[f'TOTAL_{year[-2:]}'] = tsr_by_sector[_sector_lu.values()].sum(axis=1)

    # export
    return tsr_by_sector

In [39]:
# gather annual taxable sales reports
reports = glob.glob('.\\Inputs\\*-annual-sales.xlsx')
reports = [r for r in reports if '~' not in r] # in case the spreadsheet is open

base = cities_sdf

# export to shape
for r in reports:
    tsr = process_annual_sales_to_df(r)
    base = base.merge(tsr, on='City', how='left')

base.spatial.to_featureclass(location=os.path.join(gdb,'Taxable_Sales_by_City'),sanitize_columns=False)

'e:\\Projects\\UT_Sales_Tax_Analysis\\Outputs\\Utah.gdb\\Taxable_Sales_by_City'

# Graveyard

In [ ]:
# # read in taxable sales report (excel format)
# xlsx = '.\\Inputs\\2021-annual-sales.xlsx'
# year = os.path.split(xlsx)[-1][:4]
# tsr = pd.read_excel(xlsx, sheet_name='Table 9', header=5)

In [ ]:
# # Drop last empty two rows
# tsr.drop(tsr.tail(2).index,inplace=True)

# # forward fill values from merged cells
# tsr['County'] = tsr['County'].fillna(method='ffill')
# tsr['Location Code'] = tsr['Location Code'].fillna(method='ffill')
# tsr['City'] = tsr['City'].fillna(method='ffill')

In [ ]:
# # Figures with less than 10 taxpayers have been rounded up per Tax Commission disclosure rules.
# # Rename column and convert values to boolean
# tsr.rename(columns={'Unnamed: 5':'Rounded_Up'}, inplace=True)
# crosswalk = {'*':True, np.nan:False}
# tsr['Rounded_Up'] = tsr['Rounded_Up'].map(crosswalk)

In [ ]:
# # pivot the table to get sectors as columns with tax values by city
# tsr_by_sector = pd.pivot_table(tsr,values = f'CY{year}', index ='City', columns = 'Economic Sector (NAICS Code)', 
#                                  aggfunc='first').reset_index()

# # fill NAs with 0
# tsr_by_sector = tsr_by_sector.fillna(0)

In [ ]:
# # rename sectors
# tsr_by_sector.rename(sector_lu, axis=1, inplace=True)

# # get the total sales tax revenue
# tsr_by_sector['TOTAL'] = tsr_by_sector[sector_lu.values()].sum(axis=1)
# tsr_by_sector

In [ ]:
# cities_sdf = pd.DataFrame.spatial.from_featureclass(r".\Inputs\Cities_v2.shp")
# cities_sdf = cities_sdf[['NAME','POPLASTCEN','POPLASTEST','SHAPE']].copy()
# cities_sdf.rename({'NAME':'City'}, axis=1, inplace=True)
# tsr_by_sector_sdf = cities_sdf.merge(tsr_by_sector, on='City', how='inner')
# tsr_by_sector_sdf.spatial.to_featureclass(location=os.path.join(gdb2,f'Taxable_Sales_by_City_{year}'),sanitize_columns=False)

### Calculate the differences 
*2020 - 2019 = Diff*